# SoraWatermarkCleaner - Clean Install (Virtual Environment)

This notebook uses a Python virtual environment to avoid all Colab dependency conflicts.

**Runtime:** Make sure GPU is enabled (Runtime → Change runtime type → T4 GPU)


In [ ]:
!apt-get update -qq && apt-get install -y -qq ffmpeg
!git clone https://github.com/linkedlist771/SoraWatermarkCleaner.git
%cd SoraWatermarkCleaner

## Step 2: Install with correct order

In [ ]:
# Install in specific order to avoid binary incompatibility
!pip install --no-cache-dir numpy==1.26.4
!pip install --no-cache-dir scipy==1.11.4
!pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --no-cache-dir opencv-python-headless==4.8.1.78 pillow==9.5.0
!pip install --no-cache-dir ultralytics tqdm rich typer loguru pydantic
!pip install --no-cache-dir ruptures scikit-learn pandas imageio imageio-ffmpeg
!pip install --no-cache-dir simple-lama-inpainting

# Patch the code to work without mmcv
import sys
sys.modules['mmcv'] = type(sys)('mmcv')
sys.modules['mmcv.cnn'] = type(sys)('mmcv.cnn')
sys.modules['mmcv.runner'] = type(sys)('mmcv.runner')

class MockModule:
    def __getattr__(self, name):
        return lambda *args, **kwargs: None

sys.modules['mmcv.cnn'].ConvModule = MockModule
sys.modules['mmcv.runner'].load_checkpoint = MockModule

print("mmcv mocked (LAMA-only mode)")

!pip uninstall -y ffmpeg ffmpeg-python
!pip install --no-cache-dir ffmpeg-python
print("ffmpeg-python fixed!")

## Step 3: Upload Your Video

In [ ]:
from google.colab import files
uploaded = files.upload()
video_filename = list(uploaded.keys())[0]
print(video_filename)

## Step 4: Process Video (LAMA - Fast Mode)

In [ ]:
from pathlib import Path
from sorawm.core import SoraWM
from sorawm.schemas import CleanerType

output = Path("output_clean.mp4")
sora_wm = SoraWM(cleaner_type=CleanerType.LAMA)
sora_wm.run(Path(video_filename), output)
print("Done!")

## Step 5: Download Processed Video

In [ ]:
from google.colab import files
files.download("output_clean.mp4")